In [1]:
import pandas as pd
import numpy as np

np.random.seed(2022)

In [3]:
# Data Load
ratings = pd.read_csv("ratings_small.csv")
ratings = ratings[["userId", "movieId", "rating"]]

In [4]:
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [5]:
# 다른 두 데이터를 이용해 rating 데이터의 movieId에 맞는 title에 mapping
movies = pd.read_csv("movies_metadata.csv")
links = pd.read_csv("links_small.csv")

C:\Users\sclab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# Data Preprocessing
movies = movies.fillna('')
movies = movies[movies["imdb_id"].str.startswith('tt')]
movies["imdbId"] = movies["imdb_id"].apply(lambda x: int(x[2:]))
movies = movies.merge(links, on="imdbId")

In [7]:
movies = movies[["title", "movieId"]]
movies = movies.set_index("movieId")

In [8]:
movies.head()

,title
movieId,
1,Toy Story
2,Jumanji
3,Grumpier Old Men
4,Waiting to Exhale
5,Father of the Bride Part II


In [9]:
user_movie_matrix = ratings.pivot(
    index="userId",
    columns="movieId",
    values="rating",
)
user_movie_matrix = user_movie_matrix.fillna(0)

In [10]:
user_movie_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Matrix Factorization
R = user_movie_matrix.values # 정답 R

In [12]:
n_user = R.shape[0]  # 전체 유저 수
n_item = R.shape[1]  # 전체 영화 수

In [13]:
# 잠재 요인 행렬
K = 10

In [14]:
# P와 Q를 랜덤 값으로 초기화
P = np.random.normal(size=(n_user, K))
Q = np.random.normal(size=(n_item, K))

In [15]:
P

array([[-5.27899086e-04, -2.74901425e-01, -1.39285562e-01, ...,
         5.40297269e-01,  3.73497287e-01,  3.77813394e-01],
       [-9.02131926e-02, -2.30594327e+00,  1.14276002e+00, ...,
        -8.24492228e-01,  1.89048564e-02, -3.83343556e-01],
       [-3.04185475e-01,  9.97291506e-01, -1.27273841e-01, ...,
         1.17448696e+00,  3.19068832e-01,  1.90870428e-01],
       ...,
       [ 1.56257733e+00, -1.61646773e+00,  1.01460686e+00, ...,
         1.89202195e+00, -1.74282165e-01, -5.15853812e-01],
       [-1.51614961e-01, -3.57993863e-01, -4.34264283e-01, ...,
        -1.56369174e+00,  1.41204350e+00,  5.69700971e-01],
       [-6.22145311e-01, -9.23052182e-03,  8.69726264e-02, ...,
         5.10376771e-01, -2.55302360e-01,  4.33378342e-01]])

In [16]:
Q

array([[-0.00472236,  0.66960383, -1.35039683, ..., -0.09578928,
        -1.44032777,  1.29083348],
       [-1.66968842,  2.93218199, -0.50554754, ...,  0.26717887,
         0.16262043,  0.15693477],
       [-0.29280032,  0.40258894, -1.01805777, ..., -0.85771943,
         0.67997376, -1.98131459],
       ...,
       [-1.06546485, -0.99684602,  0.48758678, ...,  1.4021354 ,
         1.28251351,  0.67603801],
       [-0.43504769,  0.48253442,  1.03926559, ..., -1.43103717,
        -0.53562373, -1.99492094],
       [ 0.66185439, -1.90742003,  0.51060278, ...,  1.9708748 ,
         1.14458556,  0.70310818]])

In [17]:
# GD를 이용한 잠재 요인 행렬 학습
# 유저 670이 영화 0에 평가한 점수를 학습
user_id = 670
item_id = 0

In [18]:
# R_hat을 계산
pred = P[user_id, :].dot(Q[item_id, :].T)
pred

0.18732828198773863

In [19]:
# R과 R_hat의 오차 계산
real = R[user_id, item_id]
real

5.0

In [20]:
error = real - pred
error

4.812671718012261

In [21]:
# GD를 이용해서 P와 Q를 업데이트
learning_rate = 0.01

In [22]:
dp = 2 * error * Q[item_id, :]
dq = 2 * error * P[user_id, :]

In [23]:
P[user_id, :] += learning_rate * dp
Q[item_id, :] += learning_rate * dq

In [24]:
P[user_id]

array([-0.62259985,  0.05522115, -0.04300771, -0.89493943, -1.16869232,
        0.25868991,  0.56970084,  0.50115672, -0.39393885,  0.5576255 ])

In [25]:
pred = P[user_id, :].dot(Q[item_id, :].T)
error = real - pred
error

3.7994326715808127

In [26]:
epochs = 10
real = R[user_id, item_id]

for epoch in range(epochs):
    pred = P[user_id, :].dot(Q[item_id, :].T)
    error = real - pred

    dp = 2 * error * Q[item_id, :]
    dq = 2 * error * P[user_id, :]

    P[user_id, :] += learning_rate * dp
    Q[item_id, :] += learning_rate * dq

    print(f"Epoch{epoch}: {round(error, 3)}")

Epoch0: 3.799
Epoch1: 2.981
Epoch2: 2.312
Epoch3: 1.769
Epoch4: 1.335
Epoch5: 0.995
Epoch6: 0.735
Epoch7: 0.538
Epoch8: 0.391
Epoch9: 0.283


In [27]:
# 전체 데이터를 이용해 P와 Q를 업데이트
K = 10

P = np.random.normal(size=(n_user, K))
Q = np.random.normal(size=(n_item, K))

epochs = 5
learning_rate = 0.01

for epoch in range(1, epochs + 1):
    total_error = 0
    iteration = 0

    # 모든 유저에 대해 반복
    for user_id in range(n_user):
        # 모든 아이템에 대해 반복 
        for item_id in range(n_item):

            real = R[user_id, item_id]

            # 평가하지 않은 경우 제외
            if real == 0:
                continue

            # P와 Q 업데이트
            pred = P[user_id, :].dot(Q[item_id, :].T)
            error = real - pred

            dp = 2 * error * Q[item_id, :]
            dq = 2 * error * P[user_id, :]

            P[user_id, :] += learning_rate * dp
            Q[item_id, :] += learning_rate * dq

            total_error += (error ** 2)
            iteration += 1

    print(f"Epoch {epoch}: {round(np.sqrt(total_error / iteration), 5)}")


Epoch 1: 3.0055
Epoch 2: 1.68267
Epoch 3: 1.3257
Epoch 4: 1.16645
Epoch 5: 1.06975


In [28]:
# 영화 추천
user_id = 124

In [29]:
# 유저 "124" 잠재요인에 모든 영화의 잠재요인을 곱해 평점 예측
prediction = P[[user_id], :].dot(Q.T)[0]

In [30]:
# 영화 아이디별 예측 평가 점수를 내림차순으로 정렬
prediction = pd.Series(
    data=prediction,
    index=user_movie_matrix.columns,
).sort_values(ascending=False)

In [31]:
prediction

movieId
1040      7.811732
27410     7.395397
32203     7.261487
136592    7.027115
80969     6.755538
            ...   
1549     -4.867292
30892    -5.280100
100517   -5.318667
64116    -5.362379
573      -5.713493
Length: 9066, dtype: float64

In [32]:
# 아직 평가하지 않은 영화만 추출
prediction = prediction[user_movie_matrix.loc[user_id] == 0]

In [33]:
# 예측 평가 점수 상위 10개의 영화 아이디 추출
recommend = prediction[:10].index

In [34]:
recommend

Int64Index([1040, 27410, 32203, 136592, 80969, 96691, 2652, 7086, 27338, 4552], dtype='int64', name='movieId')

In [35]:
movies.loc[recommend]

,title
movieId,
1040,The Secret Agent
27410,Conspiracy
32203,Brian's Song
136592,Freaky Friday
80969,Never Let Me Go
96691,Resident Evil: Retribution
2652,The Curse of Frankenstein
7086,Pygmalion
27338,The Hole
